In [1]:
import random
import os
import numpy as np
from skimage.transform import resize
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
random.seed(42)

import metpy
import datetime
import pandas as pd
import xarray as xr
import tensorflow as tf
import tensorflow_hub as hub
from PIL import Image
from pyresample import geometry, grid
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

model = hub.load("https://tfhub.dev/captain-pool/esrgan-tf2/1")
random.seed(42)

/sharedData2/nshakoor/.conda/envs/fires/lib/python3.7/site-packages/pyresample/bilinear/__init__.py:50: UserWarning: XArray and/or zarr not found, XArrayBilinearResampler won't be available.
  warnings.warn("XArray and/or zarr not found, XArrayBilinearResampler won't be available.")


In [134]:
path = '../../GOES_Files/npy_files/'
image = np.load(path + random.choice(os.listdir(path)))

## 1 Layer

In [194]:
height, width = image.shape
smallimg = resize(image, (round(height/2), round(width/2)))
control = resize(smallimg, (height, width))
mae = mean_absolute_error(image, control)
rmse = mean_squared_error(image, control, squared=False)
print(mae)
print(rmse)

0.60175735
0.8980085


## Multiple Layers

In [144]:
def load_image(array):
    """
    Returns loaded .npy file.
    
    Parameter path: Path to load .npy file from
    Precondition: path is a string
    """
    result = np.zeros((array.shape[0], array.shape[1], 3))
    result[:,:,0]= array
    result[:,:,1]= array
    result[:,:,2]= array
#     result = result.astype('uint8')
    return result

def preprocess_image(array):
    """
    Returns preprocessed input array.
    
    Parameter array: array to preprocess
    Precondition: array is a numpy array
    """
    hr_image = array
    hr_size = (tf.convert_to_tensor(hr_image.shape[:-1]) // 2) * 2
    hr_image = tf.image.crop_to_bounding_box(hr_image, 0, 0, hr_size[0], hr_size[1])
    hr_image = tf.cast(hr_image, tf.float32)
    return tf.expand_dims(hr_image, 0)

def downscale_image(image):
    """
    Returns low resolution image after scaling down input image using nearest neighbor downsampling.

    Parameter image: 3D of 4D tensor of preprocessed image
    Precondition: image is a tensor
    """
#     image_size = []
#     if len(image.shape) == 3:
#         image_size = [image.shape[1], image.shape[0]]
#     else:
#         raise ValueError("Dimension mismatch. Can work only on single image.")

# #     image = tf.squeeze(tf.cast(image, tf.uint8))
#     lr_image = np.asarray(Image.fromarray(image.numpy()).resize([image_size[0] // 2, image_size[1] // 2]))

# #     lr_image = tf.expand_dims(lr_image, 0)
# #     lr_image = tf.cast(lr_image, tf.float32)
#     return image
    height, width = image.shape
    result = np.zeros((image.shape[0]//2, image.shape[1]//2, 3))
    smallimg = resize(image, (round(height/2), round(width/2)))
    result[:,:,0] = smallimg
    result[:,:,1] = smallimg
    result[:,:,2] = smallimg
    lr_image = tf.expand_dims(result, 0)
    lr_image = tf.cast(lr_image, tf.float32)
    return lr_image

In [193]:
hr_image = preprocess_image(load_image(image))
lr_image = downscale_image(image)
fake_image = model(lr_image)
fake_image = tf.squeeze(fake_image)

hr_image = tf.squeeze(hr_image).numpy()
lr_image = tf.squeeze(lr_image).numpy()
lr_image = resize(lr_image, (hr_image.shape[0], hr_image.shape[1], 3)).ravel()
# lr_image[:,:,0] = resize(lr_image[:,:,0], (hr_image.shape[0], hr_image.shape[1]))
# lr_image = tf.squeeze(lr_image).numpy().resize([hr_image.shape[0], hr_image.shape[1]])[:,:,0]
# # lr_image = np.asarray(Image.fromarray(tf.squeeze(lr_image).numpy()).resize([hr_image.shape[0], hr_image.shape[1]]))[:,:,0]
fake_image = resize(fake_image.numpy(), (hr_image.shape[0], hr_image.shape[1], 3)).ravel()
hr_image = hr_image.ravel()

control_mae = mean_absolute_error(hr_image, lr_image)
control_rmse = mean_squared_error(hr_image, lr_image, squared=False)

mae = mean_absolute_error(hr_image, fake_image)
rmse = mean_squared_error(hr_image, fake_image, squared=False)
print(control_mae)
print(mae)
print(rmse)

0.60175735
1.5496775
1.9424095


In [177]:
lr_image.shape

(500, 500, 3)

In [188]:
fake_image.shape

(500, 500, 3)

In [189]:
hr_image.shape

(500, 500, 3)

In [168]:
lr_image.shape

AttributeError: 'NoneType' object has no attribute 'shape'

In [89]:
hr_image.shape

TensorShape([1, 500, 500, 3])

In [152]:
fake_image[:,:,0]

<tf.Tensor: shape=(1000, 1000), dtype=float32, numpy=
array([[169.73962, 239.87773, 276.17017, ..., 268.43512, 257.08893,
        249.33235],
       [249.64377, 268.67477, 269.13455, ..., 268.24387, 283.46872,
        261.65567],
       [274.6035 , 288.28812, 273.34018, ..., 261.06485, 275.35196,
        252.22685],
       ...,
       [267.17377, 282.70065, 279.44284, ..., 285.3578 , 279.8649 ,
        287.02298],
       [266.06888, 279.07535, 261.0966 , ..., 293.8519 , 291.835  ,
        288.44467],
       [259.67664, 272.0229 , 263.03055, ..., 296.18527, 292.31326,
        280.96405]], dtype=float32)>

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15, 5))

x = ax[0].imshow(, cmap='hot', aspect='equal')
ax[0].set_title('HF = HR – LR')
ax[0].axis('off')

In [14]:
pic = fake_image[0,:,:,0].numpy()

In [59]:
lr_image.shape

TensorShape([500, 500, 3])

In [104]:
lr_image[:,:]

array([[240, 240, 241, ...,   4,   4,   4],
       [240, 240, 241, ...,   4,   4,   4],
       [239, 240, 242, ...,   4,   4,   4],
       ...,
       [ 31,  31,  30, ...,   6,   5,   5],
       [ 30,  30,  30, ...,   4,   3,   3],
       [ 30,  30,  30, ...,   3,   2,   2]], dtype=uint8)

In [15]:
fake_image[0,:,:,0]

<tf.Tensor: shape=(1000, 1000), dtype=float32, numpy=
array([[147.21501  , 208.58575  , 241.5658   , ...,   3.406515 ,
          2.948714 ,   3.010914 ],
       [215.34886  , 230.84985  , 231.39037  , ...,   3.030803 ,
          2.9688818,   2.860043 ],
       [236.14165  , 246.19382  , 234.54842  , ...,   3.09882  ,
          2.9988825,   2.8757493],
       ...,
       [ 28.99264  ,  30.568092 ,  30.410713 , ...,   1.1022397,
          1.6600069,   1.7966036],
       [ 29.3202   ,  30.831007 ,  28.597977 , ...,  -2.5142295,
         -1.0270957,   0.5452343],
       [ 29.335653 ,  30.954165 ,  29.363674 , ...,  -2.5281239,
         -1.6440666,   1.4088746]], dtype=float32)>

In [50]:
image

array([[245.86417, 240.95888, 240.95888, ..., 260.64215, 260.64215,
        262.04288],
       [237.21982, 239.18741, 243.07973, ..., 260.2246 , 260.84793,
        260.84793],
       [237.21982, 239.18741, 243.07973, ..., 260.2246 , 260.84793,
        259.79868],
       ...,
       [286.79688, 286.65906, 286.65906, ..., 261.0518 , 261.6519 ,
        260.2246 ],
       [286.451  , 286.93393, 287.4753 , ..., 259.5825 , 259.5825 ,
        258.00385],
       [286.79688, 286.38123, 286.3113 , ..., 259.5825 , 258.00385,
        258.23672]], dtype=float32)